In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from xml.etree import ElementTree as ET
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
TF_AVAILABLE = True

In [ ]:
from pathlib import Path

DATA_DIR = Path('/kaggle/input/face-mask-detection')
IMAGES_DIR = DATA_DIR / 'images'
ANNOTS_DIR = DATA_DIR / 'annotations'

print("\nLooking for dataset folders under:", DATA_DIR.resolve())
for p in [IMAGES_DIR, ANNOTS_DIR]:
    print(f" - {p} -> exists? {p.exists()}")


In [ ]:
import pandas as pd
from pathlib import Path

def parse_voc_xml(xml_path):
    """
    Parse a single Pascal VOC XML file.
    Returns list of dicts: [{'filename':..., 'xmin':..., 'ymin':..., 'xmax':..., 'ymax':..., 'class':...}, ...]
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()
    filename = root.findtext('filename')
    objects = []
    for obj in root.findall('object'):
        cls = obj.findtext('name')
        bndbox = obj.find('bndbox')
        xmin = int(float(bndbox.findtext('xmin')))
        ymin = int(float(bndbox.findtext('ymin')))
        xmax = int(float(bndbox.findtext('xmax')))
        ymax = int(float(bndbox.findtext('ymax')))
        objects.append({
            'filename': filename,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax,
            'class': cls
        })
    return objects

# Collect all annotations
annots_dir = ANNOTS_DIR
records = []
if annots_dir.exists():
    xml_files = sorted(list(annots_dir.glob("*.xml")))
    print(f"Found {len(xml_files)} XML files in {annots_dir}")
    for xml in xml_files:
        records.extend(parse_voc_xml(xml))
else:
    print(f"Annotations folder not found: {annots_dir}")
    
# Create DataFrame
df = pd.DataFrame(records, columns=['filename','xmin','ymin','xmax','ymax','class'])
print("DataFrame shape:", df.shape)
display(df.head())

# Basic checks
print("\nMissing values per column:")
print(df.isnull().sum())

# Count unique images & per-class counts
if not df.empty:
    print("\nUnique images:", df['filename'].nunique())
    print("Objects per class:")
    print(df['class'].value_counts())
else:
    print("No annotation records were parsed — check your annotations folder and XML format.")


In [ ]:
invalid = df[(df['xmax'] <= df['xmin']) | (df['ymax'] <= df['ymin'])]
print(f" Invalid boxes found: {len(invalid)}")


In [ ]:
import matplotlib.pyplot as plt

class_counts = df['class'].value_counts()

plt.figure(figsize=(8,5))
plt.bar(class_counts.index, class_counts.values)
plt.title("Class Distribution (Number of Objects per Class)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
plt.figure(figsize=(6,6))
plt.pie(class_counts.values,
        labels=class_counts.index,
        autopct='%1.1f%%',
        startangle=140,
        colors=["#66c2a5", "#fc8d62", "#8da0cb"])
plt.title("Class Distribution (%)")
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt

def show_image_with_boxes(filename):
    img_path = os.path.join(IMAGES_DIR, filename)
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    boxes = df[df['filename'] == filename]
    
    for _, row in boxes.iterrows():
        cv2.rectangle(image, (row['xmin'], row['ymin']), (row['xmax'], row['ymax']),
                      (0, 255, 0), 2)
        cv2.putText(image, row['class'], (row['xmin'], row['ymin'] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    plt.figure(figsize=(8,8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

show_image_with_boxes(df['filename'].iloc[0])


In [ ]:
import cv2
import matplotlib.pyplot as plt
import random
import os

sample_files = random.sample(list(df['filename'].unique()), 5)

def show_image_with_boxes(filename):
    img_path = os.path.join(IMAGES_DIR, filename)
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    boxes = df[df['filename'] == filename]
    
    for _, row in boxes.iterrows():
        color = (0, 255, 0) if row['class'] == 'with_mask' else \
                (255, 0, 0) if row['class'] == 'without_mask' else \
                (255, 165, 0)
        cv2.rectangle(image, (row['xmin'], row['ymin']), (row['xmax'], row['ymax']), color, 2)
        cv2.putText(image, row['class'], (row['xmin'], row['ymin'] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    plt.figure(figsize=(6,6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(filename)
    plt.show()

for file in sample_files:
    show_image_with_boxes(file)


In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

IMG_SIZE = 128  
faces = []
labels = []

print("Starting to load and crop images...")
for idx, row in df.iterrows():
    img_path = IMAGES_DIR / row['filename']
    if not img_path.exists():
        continue 
    
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    
    x1, y1, x2, y2 = row['xmin'], row['ymin'], row['xmax'], row['ymax']
    face = img[y1:y2, x1:x2]
    
    # skip invalid boxes
    if face.size == 0:
        continue
    
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
    face = preprocess_input(face)
    
    faces.append(face)
    labels.append(row['class'])

print(f"Loaded {len(faces)} cropped faces.")

X = np.array(faces, dtype="float32")
y = np.array(labels)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

print("Classes found:", le.classes_)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, stratify=y_cat, random_state=42
)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
print(f"X shape: {X_train.shape[1:]}")


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

INIT_LR = 1e-4     
EPOCHS = 10       
BS = 32            

baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(128, 128, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])


In [ ]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
print("[INFO] Training the model...")
H = model.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    steps_per_epoch=len(X_train) // BS,
    validation_data=(X_test, y_test),
    validation_steps=len(X_test) // BS,
    epochs=EPOCHS
)

In [ ]:
print("[INFO] Evaluating the model...")
predIdxs = model.predict(X_test, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)

y_test_labels = np.argmax(y_test, axis=1)

print(classification_report(y_test_labels, predIdxs,
                            target_names=['mask_incorrect', 'with_mask', 'without_mask']))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
model.save("mask_detector_v1.h5")
print(" Model saved as mask_detector_v1.h5")


In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml


In [ ]:
import os
print(os.path.exists("haarcascade_frontalface_default.xml"))
